# Chicago Crime Data
- Source: Chicago Data Portal: Crimes 2001 to Present
  - Data Description:
  - All Crimes that were reported in the city of Chicago and their details

Includes:
- type of crime, exact date/time, lat/long, District/ward, was there an arrest, etc.
  - Note: the .csv is very large and needs to be processed into smaller files to fit inside a GitHub repository. We have provided a helper notebook below to process your downloaded file into a repo-friendly format.
- Data Processing Helper Notebook:
  - Use the helper notebook in this repository to process your manually-downloaded csv into several .csv.gz files: https://github.com/coding-dojo-data-science/preparing-chicago-crime-data
  - You can either Fork your own copy of the repository and rename it to be your "Project 4" repository, or you can download the notebook and save it in your own repository.
- Supplemental Data: Holiday Data
  - To answer some of the possible questions, you may need to perform some feature engineering, like adding holiday information from an API.
  - We have provided an extract of holidays, but you may wish to extract additional holiday information for dates not included in this file.

    - Raw CSV URL (Pandas): https://docs.google.com/spreadsheets/d/e/2PACX-1vTMR4vi1nFtbYO_WLqecU6WoRAQ0F-_QCMcuWUoZhuTygAP2U668cddCWBeAV2eop9fFdyXX856AB2j/pub?output=csv
- Notes/Considerations:
  - You may need to keep 2 forms of the data:
  - The original individual crime data with a datetime index. (Each row is 1 crime)
  - A resampled/converted crime counts version (Each row is 1 day)

## Task
Our task is to answer a series of questions about trends in crimes in Chicago for a reporter for the local newspaper.

- Stakeholder Questions to Answer (pick at least 3 topics) Select 3 or more of the following topics to analyze:
1). Comparing Police Districts:
  - Which district has the most crimes? Which has the least?
  
2). Crimes Across the Years:
  - Is the total number of crimes increasing or decreasing across the years?
  - Are there any individual crimes that are doing the opposite (e.g decreasing when overall crime is increasing or vice-versa)?
  
3). Comparing AM vs. PM Rush Hour:
  - Are crimes more common during AM rush hour or PM rush hour?
    - You can consider any crime that occurred between 7 AM - 10 AM as AM rush hour
    - You can consider any crime that occurred between 4 -7 PM as PM rush hour.
  - What are the top 5 most common crimes during AM rush hour? What are the top 5 most common crimes during PM rush hour?
  - Are Motor Vehicle Thefts more common during AM rush hour or PM Rush Hour?
  
4). Comparing Months:
 - What months have the most crime? What months have the least?
 - Are there any individual crimes that do not follow this pattern? If so, which crimes?
 
5). Comparing Holidays:
  - Are there any holidays that show an increase in the # of crimes?
  - Are there any holidays that show a decrease in the # of crimes?